In [ ]:
%%writefile flop.sv

module flop (
    input clk, rst_n,
    input [3:0] data_in, 

    output logic [3:0] data_out
);

always_ff @( posedge clk or negedge rst_n ) begin
    if (!rst_n)
        data_out <= 'b0;
    else
        data_out <= data_in;
end

endmodule



In [ ]:
%%writefile Makefile

# Verilator example makefile
# Norbertas Kremeris 2021
MODULE=flop

.PHONY:sim
sim: waveform.vcd

.PHONY:verilate
verilate: .stamp.verilate

.PHONY:build
build: obj_dir/V${MODULE}

.PHONY:waves
waves: waveform.vcd
	@echo
	@echo "### WAVES ###"
	gtkwave waveform.vcd -a gtkwave_setup.gtkw

waveform.vcd: ./obj_dir/V$(MODULE)
	@echo
	@echo "### SIMULATING ###"
	./obj_dir/V$(MODULE) +verilator+rand+reset+1

./obj_dir/V$(MODULE): .stamp.verilate
	@echo
	@echo "### BUILDING SIM ###"
	make -C obj_dir -f V$(MODULE).mk V$(MODULE)

.stamp.verilate: $(MODULE).sv tb_$(MODULE).cpp
	@echo
	@echo "### VERILATING ###"
	verilator -Wall --trace --x-assign unique --x-initial unique -cc $(MODULE).sv --exe tb_$(MODULE).cpp \
		-CFLAGS "`python3-config --cflags`" -LDFLAGS "`python3-config --ldflags`"
	@touch .stamp.verilate

.PHONY:lint
lint: $(MODULE).sv
	verilator --lint-only $(MODULE).sv

.PHONY: clean
clean:
	rm -rf .stamp.*;
	rm -rf ./obj_dir
	rm -rf waveform.vcd


In [ ]:
%%writefile tb_flop.cpp

#include <stdlib.h>
#include <iostream>
#include <cstdlib>
#include <verilated.h>
#include <verilated_vcd_c.h>
#include "Vflop.h"
#include "Vflop___024root.h"

#include <pybind11/pybind11.h>
namespace py = pybind11;

#define MAX_SIM_TIME 36
#define VERIF_START_TIME 7
vluint64_t sim_time = 0;

int main(int argc, char** argv, char** env) {
    srand (time(NULL));
    Verilated::commandArgs(argc, argv);
    Vflop *dut = new Vflop;
    
    Verilated::traceEverOn(true);
    VerilatedVcdC *m_trace = new VerilatedVcdC;
    dut->trace(m_trace, 5);
    m_trace->open("waveform.vcd");

    dut->rst_n = 0;
    dut->clk   = 0;

    while (sim_time < MAX_SIM_TIME) {
        dut->clk ^= 1;
        dut->eval();
        printf("sim_time, rst_n is %ld, %d\n", sim_time, dut->rst_n);

        if (sim_time > 3 && sim_time < 7) {
            dut->rst_n = 0;
        } 
        else {
            dut->rst_n = 1;
            if (dut->clk == 1)
                dut->data_in++;
        };

        m_trace->dump(sim_time);
        sim_time++;
    }


    m_trace->close();
    delete dut;
    exit(EXIT_SUCCESS);
}





In [ ]:
!make clean verilate build waves